# ZMS LAB 03 - case RIG



## Opis zajęć

Kilkoro świeżo upieczonych absolwentów SGH, po uzyskaniu licencji doradcy inwestycyjnego, założyło spółkę RIG i otworzyło własny zamknięty fundusz inwestycyjny. 

Strategia funduszu zakłada inwestowanie w portfel akcji spośród 11 najbardziej płynnych papierów kwotowanych na GPW. 

Celem jest maksymalizacja stopy zwrotu w horyzoncie 1 roku (252 dni sesyjne) przy założeniu, że:

- Wariant 1. Oczekiwana stopa zwrotu nie może być niższa niż 5%.

- Wariant 2. Wariancja portfela nie może być wyższa niż połowa wariancji stóp zwrotu z akcji o najwyższej wariancji



## ROZWIĄZANIE

### 1. Akcje - dane

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sc
import scipy.optimize as so
import statsmodels.formula.api as sm

In [ ]:
akcje = pd.read_csv("akcje.txt", delimiter = ";", header = None)
del akcje[0]
akcje.head()
akcje.describe()

In [ ]:
akcje.plot()
akcje.drop([3,4], axis = 1).plot();

#### Akcje - zwroty i stopy zwrotów

In [ ]:
akcje_zwrot = akcje.apply(lambda x: x/x[0])
akcje_zwrot.head(10)
akcje_zwrot.tail(10)

In [ ]:
akcje_zwrot.plot()
akcje_zwrot.drop([4, 5, 11], axis = 1).plot();

In [ ]:
akcje_stopa = akcje.apply(lambda x: np.log(x) - np.log(x.shift(1)))
akcje_stopa.plot()
akcje_stopa.drop([6,8], axis = 1).plot();

In [ ]:
akcje_stopa.iloc[:, 10].hist(bins = 20);

#### Akcje - średnie ruchome i prosty model MNK

In [ ]:
ma20  = akcje.rolling(window = 20, center = False).mean()
ma50  = akcje.rolling(window = 50, center = False).mean()
ma200  = akcje.rolling(window = 200, center = False).mean()

In [ ]:
x = np.arange(len(akcje))
y = akcje[3]

model = sm.ols("y ~ x", data = akcje)
model = model.fit()
model.summary()

In [ ]:
trendline = model.predict(akcje)

plt.plot(akcje[3], label = "cena")
plt.plot(ma50[3], label = "ma50")
plt.plot(ma200[3], label = "ma200")
plt.plot(trendline, label = "trend")
plt.legend()
plt.show()

### 2. Wybór optymalnego portfela na podstawie 2 scenariuszy

#### WARIANT 1: oczekiwana stopa wzrotu nie niższa niz 5%

chcemy aby oczekiwana stopa zwrotu wyniosła co najmniej 5%, za pomocą średniej geometrycznej wyliczamy jaka powinna być jej dzienna wartość

In [ ]:
# 1 rok = 252 dni sesyjne
r_min = 1.05 ** (1/252) - 1

#### WARIANT 2: wariancja portfela nie może być wyższa niż połowa wariancji stóp zwrotu z akcji o najwyższej wariancji

wyliczamy średnią i odchylenie std stopy zwrotu dla każdego papieru i na tej podstawie wyznaczamy ograniczenie


In [ ]:
r_mean = akcje_stopa.mean()
r_std = akcje_stopa.std()

# ograniczenie wariancji:
var_max = 0.5*(max(r_std)**2)

#### Bazowy portfel funduszu

- tworzymy macierze kowariancji i korelacji na podstawie stop zwrotu 
- przyjmujemy początkowe wagi dla portfela
- funkcje obliczające średni zwrot z portfela i wariancję portfela

(por. np. https://www.bankier.pl/wiadomosc/Dywersyfikacja-7337529.html)

In [ ]:
cov = akcje_stopa.cov()
cor = akcje_stopa.corr()

wagi = np.ones(akcje.shape[1])/akcje.shape[1]

In [ ]:
def portfolio_mean(wagi):
    return sum(wagi * r_mean) 

def portfolio_mean_neg(wagi):
    return -sum(wagi * r_mean)

def portfolio_var(wagi):
    return np.dot(np.dot(wagi, cov), wagi)

print(portfolio_mean(wagi))
print(portfolio_mean_neg(wagi))
print(portfolio_var(wagi))

#### Modele optymalizacyjne dla każdego scenariusza

[COBYLA](https://en.wikipedia.org/wiki/COBYLA) = Constrained optimization by linear approximation

type: inequality means that the constraint function result is to be non-negative

#### WARIANT 1 - warunki ograniczajace dla modelu optymlizacyjnego

In [ ]:
cons1 = ({'type':'ineq', 'fun': lambda wagi: sum(wagi) - 1},
        {'type':'ineq', 'fun': lambda wagi: portfolio_mean(wagi) - r_min})

# model optymalizacyjny 1
opt_1 = so.minimize(portfolio_var, wagi, method = "COBYLA", constraints = cons1)

# wyniki
wagi_1 = opt_1.x
print(wagi_1)
print("Suma wag: ",sum(wagi_1))
print("Średnia stopa zwrotu: ",(1 + portfolio_mean(wagi_1)) ** 252 - 1 )
print("Wariancja portfela: ",portfolio_var(wagi_1) )

#### WARIANT 2 - warunki ograniczajace dla modelu optymlizacyjnego

In [ ]:
cons2 = ({'type':'ineq', 'fun': lambda wagi: sum(wagi) - 1},
         {'type':'ineq', 'fun': lambda wagi: -sum(wagi) + 1},
        {'type':'ineq', 'fun': lambda wagi: var_max - portfolio_var(wagi)})

# model optymalizacyjny 1
opt_2 = so.minimize(portfolio_mean_neg, wagi, method = "SLSQP",
    bounds = [(0,1) for i in range(len(wagi))], constraints = cons2)

# wyniki
wagi_2 = opt_2.x
print(wagi_2) 
print("Suma wag: ",sum(wagi_2))
print("Średnia stopa zwrotu: ", (1 + portfolio_mean(wagi_2)) ** 252 - 1 )
print("Wariancja portfela: ", portfolio_var(wagi_2) )

### 3. Model symulacyjny i kod wywołania

tworzymy symulację, która sprawdzi czy faktycznie dla tak zoptymalizowanego modelu fundusz osiągnie zakładany zysk

przydatne linki:

- [Rozkład Choleskiego](https://pl.wikipedia.org/wiki/Rozk%C5%82ad_Choleskiego)
- [Jak losować zmienne ze złożonych rozkładów?](http://pbiecek.github.io/Przewodnik/Programowanie/generatory_3.html)

In [ ]:
def symulacja (data_set, horyzont, w):
    data_set_r = data_set.apply(lambda x: np.log(x) - np.log(x.shift(1)))
    std_r = data_set_r.std()
    mean_r = data_set_r.mean()
    corr = data_set_r.corr()
    L = np.linalg.cholesky(corr)
    
    # początkowa wartość udziału:
    P0 = data_set.iloc[len(data_set) - 1] 
    
    # generujemy zwroty zakładając wielowymiarowy rozkład normalny
    rates = []
    for i in range(horyzont):
        los = sc.random.normal(0, 1, len(data_set.iloc[1]))
        los = np.dot(los, np.transpose(L))
        rates.append(los * std_r + mean_r)
    
    r_cum = list(map(sum, np.transpose(rates)))
    r_cum = list(map(np.exp, r_cum))
    V = sum(w * P0 * r_cum)
    return(V)


def uruchomienie(symul, data_set, horyzont, w):
    V = []    
    for i in range(symul):
        V.append(symulacja(data_set, horyzont, w))
    return(V)

### 3. Symulacje i wyniki

#### WARIANT 1

In [ ]:
wart_pocz1 = sum(akcje.iloc[-1] * wagi_1)
wart_kon1 = uruchomienie(100, akcje, 252, wagi_1)

In [ ]:
prc_rt1 = np.array([(w - wart_pocz1)/wart_pocz1*100 for w in wart_kon1])
print(np.mean(prc_rt1))
print(np.min(prc_rt1),np.max(prc_rt1))

In [ ]:
plt.hist(prc_rt1, bins=10)
plt.xlabel('Stopa zwrotu (%)');

#### WARIANT 2

In [ ]:
wart_pocz2 = sum(akcje.iloc[-1] * wagi_2)
wart_kon2 = uruchomienie(100, akcje, 252, wagi_2)

In [ ]:
prc_rt2 = np.array([(w - wart_pocz2)/wart_pocz2*100 for w in wart_kon2])
print(np.mean(prc_rt2))
print(np.min(prc_rt2),np.max(prc_rt2))

In [ ]:
plt.hist(prc_rt2, bins=10, color="green")
plt.xlabel('Stopa zwrotu (%)');